In [1]:
import pandas as pd

In [6]:
czk_df = pd.read_parquet('FRED_Req.par')
covid_df = pd.read_parquet('OWID_Req.csv')

In [7]:
czk_df

,date,value
0,1991-01-01,27.646818
1,1991-02-01,27.244000
2,1991-03-01,28.740000
3,1991-04-01,29.943333
4,1991-05-01,30.115714
...,...,...
359,2020-12-01,21.629913
360,2021-01-01,21.474714
361,2021-02-01,21.388100
362,2021-03-01,22.002043


In [8]:
#make df from FRED data, convert index to datetime
czk_df.index = pd.to_datetime(czk_df.index)
czk_df = czk_df.loc['2020-03-01':,:] #this does work but with warning
czk_df.columns = ['CZK/USD']
czk_df['CZK/USD'] = czk_df['CZK/USD'].astype(float)
#df

ValueError: Length mismatch: Expected axis has 2 elements, new values have 1 elements

In [ ]:
#cut data on Czech Republic, drop empty columns and rows, convert index to datetime
covid_df = covid_df.where(covid_df['location'] == 'Czechia').dropna(how='all')
covid_df = covid_df.dropna(axis=1, how='all')
covid_df.index = pd.to_datetime(covid_df.index)
#covid_df

In [ ]:
#problem: FRED data monthly, OWID data daily -> need to convert OWID data
#add up cases of a month, save in a list
list_monthly_cases = []
list_monthly_cases_index = []
monthly_cases = covid_df.iloc[0]['new_cases']
for i in range(len(covid_df)-1):
    if covid_df.index[i].month == covid_df.index[i+1].month:
        monthly_cases = monthly_cases + covid_df.iloc[i+1]['new_cases']
    else:
        list_monthly_cases.append(monthly_cases)
        list_monthly_cases_index.append(covid_df.index[i+1])
        monthly_cases = covid_df.iloc[i+1]['new_cases']

In [ ]:
'''
#control if idea works -> it does
print(list_monthly_cases)
print(covid_df.loc['2020-04-30']['total_cases']-covid_df.loc['2020-03-31']['total_cases'])
covid_df.loc['2020-03-31']['total_cases']
'''

In [ ]:
#safe list of monthly cases as df, concat with df on CZK/USD data
monthly_cases_summed = pd.DataFrame(data = list_monthly_cases, index = list_monthly_cases_index, columns = ['Summed Monthly Cases'])
result = pd.concat([czk_df,monthly_cases_summed], axis = 1)
result

In [ ]:
result.to_csv('merged_data.csv')